In [3]:
#import des bibliothèques
import numpy as np
import pandas as pd #parcequ'on est pas des PD nous
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode
import json
import geojson
import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox
from urllib.request import urlopen
from datetime import datetime
import statistics
init_notebook_mode(connected=True)

In [3]:
#import de l'ensemble des données
#df_2017 = pd.read_csv('valeursfoncieres-2017.txt', sep="|",low_memory=False,decimal=',')
#df_2018 = pd.read_csv('valeursfoncieres-2018.txt', sep="|",low_memory=False,decimal=',')
#df_2019 = pd.read_csv('valeursfoncieres-2019.txt', sep="|",low_memory=False,decimal=',')
df_2020 = pd.read_csv('valeursfoncieres-2020.txt', sep="|",low_memory=False,decimal=',')
#df_2021 = pd.read_csv('valeursfoncieres-2021.txt', sep="|",low_memory=False,decimal=',')

In [ ]:
#affichage des 5 premiers élément du dataframe 2017
#df_2017.head()

In [4]:
#petite fonction qui permet de vérifier si une colonne ne contient aucun élément et dans ce cas cette dernière est supprimé
def ColonneVide(df):
    nunique = df.nunique()
    compteur = 0
    for i in df.columns:
        if nunique[compteur] == 0:
            del df[i]
        compteur += 1
    return df

In [5]:
#on supprime les colonnes vides
#df_2017 = ColonneVide(df_2017)
#df_2018 = ColonneVide(df_2018)
#df_2019 = ColonneVide(df_2019)
df_2020 = ColonneVide(df_2020)
#df_2021 = ColonneVide(df_2021)
df_2020.head(10)

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,...,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,1,07/01/2020,Vente,8000.0,NaN,NaN,NaN,B063,FORTUNAT,1250.0,...,NaN,NaN,0,NaN,NaN,NaN,NaN,T,NaN,1061.0
1,1,02/01/2020,Vente,2175.0,NaN,NaN,NaN,B124,TERRES DES CINQ SAULES,1290.0,...,NaN,NaN,0,NaN,NaN,NaN,NaN,BT,NaN,85.0
2,1,02/01/2020,Vente,2175.0,NaN,NaN,NaN,B006,BOIS DU CHAMP RION,1290.0,...,NaN,NaN,0,NaN,NaN,NaN,NaN,T,NaN,1115.0
3,1,02/01/2020,Vente,2175.0,NaN,NaN,NaN,B025,EN COROBERT,1290.0,...,NaN,NaN,0,NaN,NaN,NaN,NaN,T,NaN,1940.0
4,1,02/01/2020,Vente,2175.0,NaN,NaN,NaN,B124,TERRES DES CINQ SAULES,1290.0,...,NaN,NaN,0,NaN,NaN,NaN,NaN,T,NaN,1148.0
5,1,02/01/2020,Vente,2175.0,NaN,NaN,NaN,B124,TERRES DES CINQ SAULES,1290.0,...,NaN,NaN,0,NaN,NaN,NaN,NaN,T,NaN,2960.0
6,1,07/01/2020,Vente,75000.0,NaN,NaN,RUE,0064,DE LA CHARTREUSE,1960.0,...,NaN,NaN,0,NaN,NaN,NaN,NaN,AB,NaN,610.0
7,1,07/01/2020,Vente,123.0,NaN,NaN,NaN,B041,CHAMP PORTIER,1370.0,...,NaN,NaN,0,NaN,NaN,NaN,NaN,S,NaN,55.0
8,1,07/01/2020,Vente,123.0,NaN,NaN,NaN,B041,CHAMP PORTIER,1370.0,...,NaN,NaN,0,NaN,NaN,NaN,NaN,S,NaN,68.0
9,1,09/01/2020,Vente,72000.0,NaN,NaN,NaN,B034,CHAMPEL,1270.0,...,NaN,NaN,0,NaN,NaN,NaN,NaN,J,NaN,328.0


In [5]:
#variables utiles pour la suite

REGIONS = {
    'Auvergne-Rhône-Alpes': ['01', '03', '07', '15', '26', '38', '42', '43', '63', '69', '73', '74'],
    'Bourgogne-Franche-Comté': ['21', '25', '39', '58', '70', '71', '89', '90'],
    'Bretagne': ['35', '22', '56', '29'],
    'Centre-Val de Loire': ['18', '28', '36', '37', '41', '45'],
    'Corse': ['2A', '2B'],
    'Grand Est': ['08', '10', '51', '52', '54', '55', '57', '67', '68', '88'],
    'Guadeloupe': ['971'],
    'Guyane': ['973'],
    'Hauts-de-France': ['02', '59', '60', '62', '80'],
    'Île-de-France': ['75', '77', '78', '91', '92', '93', '94', '95'],
    'La Réunion': ['974'],
    'Martinique': ['972'],
    'Normandie': ['14', '27', '50', '61', '76'],
    'Nouvelle-Aquitaine': ['16', '17', '19', '23', '24', '33', '40', '47', '64', '79', '86', '87'],
    'Occitanie': ['09', '11', '12', '30', '31', '32', '34', '46', '48', '65', '66', '81', '82'],
    'Pays de la Loire': ['44', '49', '53', '72', '85'],
    'Provence-Alpes-Côte d\'Azur': ['04', '05', '06', '13', '83', '84'],
}

DEPARTMENTS = {
    '01': 'Ain', 
    '02': 'Aisne', 
    '03': 'Allier', 
    '04': 'Alpes-de-Haute-Provence', 
    '05': 'Hautes-Alpes',
    '06': 'Alpes-Maritimes', 
    '07': 'Ardèche', 
    '08': 'Ardennes', 
    '09': 'Ariège', 
    '10': 'Aube', 
    '11': 'Aude',
    '12': 'Aveyron', 
    '13': 'Bouches-du-Rhône', 
    '14': 'Calvados', 
    '15': 'Cantal', 
    '16': 'Charente',
    '17': 'Charente-Maritime', 
    '18': 'Cher', 
    '19': 'Corrèze', 
    '2A': 'Corse-du-Sud', 
    '2B': 'Haute-Corse',
    '21': 'Côte-d\'Or', 
    '22': 'Côtes-d\'Armor', 
    '23': 'Creuse', 
    '24': 'Dordogne', 
    '25': 'Doubs', 
    '26': 'Drôme',
    '27': 'Eure', 
    '28': 'Eure-et-Loir', 
    '29': 'Finistère', 
    '30': 'Gard', 
    '31': 'Haute-Garonne', 
    '32': 'Gers',
    '33': 'Gironde', 
    '34': 'Hérault', 
    '35': 'Ille-et-Vilaine', 
    '36': 'Indre', 
    '37': 'Indre-et-Loire',
    '38': 'Isère', 
    '39': 'Jura', 
    '40': 'Landes', 
    '41': 'Loir-et-Cher', 
    '42': 'Loire', 
    '43': 'Haute-Loire',
    '44': 'Loire-Atlantique', 
    '45': 'Loiret', 
    '46': 'Lot', 
    '47': 'Lot-et-Garonne', 
    '48': 'Lozère',
    '49': 'Maine-et-Loire', 
    '50': 'Manche', 
    '51': 'Marne', 
    '52': 'Haute-Marne', 
    '53': 'Mayenne',
    '54': 'Meurthe-et-Moselle', 
    '55': 'Meuse', 
    '56': 'Morbihan', 
    '57': 'Moselle', 
    '58': 'Nièvre', 
    '59': 'Nord',
    '60': 'Oise', 
    '61': 'Orne', 
    '62': 'Pas-de-Calais', 
    '63': 'Puy-de-Dôme', 
    '64': 'Pyrénées-Atlantiques',
    '65': 'Hautes-Pyrénées', 
    '66': 'Pyrénées-Orientales', 
    '67': 'Bas-Rhin', 
    '68': 'Haut-Rhin', 
    '69': 'Rhône',
    '70': 'Haute-Saône', 
    '71': 'Saône-et-Loire', 
    '72': 'Sarthe', 
    '73': 'Savoie', 
    '74': 'Haute-Savoie',
    '75': 'Paris', 
    '76': 'Seine-Maritime', 
    '77': 'Seine-et-Marne', 
    '78': 'Yvelines', 
    '79': 'Deux-Sèvres',
    '80': 'Somme', 
    '81': 'Tarn', 
    '82': 'Tarn-et-Garonne', 
    '83': 'Var', 
    '84': 'Vaucluse', 
    '85': 'Vendée',
    '86': 'Vienne', 
    '87': 'Haute-Vienne', 
    '88': 'Vosges', 
    '89': 'Yonne', 
    '90': 'Territoire de Belfort',
    '91': 'Essonne', 
    '92': 'Hauts-de-Seine', 
    '93': 'Seine-Saint-Denis', 
    '94': 'Val-de-Marne', 
    '95': 'Val-d\'Oise',
    '971': 'Guadeloupe', 
    '972': 'Martinique', 
    '973': 'Guyane', 
    '974': 'La Réunion', 
    '976': 'Mayotte',
}

with urlopen('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements-version-simplifiee.geojson') as response:
  departements_france = json.load(response)

with urlopen('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/regions-version-simplifiee.geojson') as response:
  regions_france = json.load(response)

with urlopen('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/communes-version-simplifiee.geojson') as response:
  communes_france = json.load(response)

In [7]:
def find_key(dico,v): 
    for k, val in dico.items(): 
        if v in val: 
            return k 

In [6]:
#ajout des régions dans le dataframe
df_2020["Regions"] = None

tab = []
for i in df_2020["Code departement"]:
    tab.append(find_key(REGIONS, i))
df_2020["Regions"] = tab

NameError: name 'df_2020' is not defined

In [8]:
#Calcul du prix au m², du rapport valeurs fonctières/nombre de piece des maisons et appartement
#on met aussi a jour les noms des villes pour qu'ils correspondent au format des gesjson
df_2020['Commune'] = df_2020['Commune'].apply(lambda x: x[:1] + x[1:].lower())
df_2020["Prix au m2"] = round(df_2020["Valeur fonciere"]/df_2020["Surface terrain"])
df_2020["Rapport valeurs foncières / nombre de pieces"] = round(df_2020["Valeur fonciere"]/df_2020["Nombre pieces principales"])
Maison_mask = df_2020['Type local']=='Maison'
Appart_mask = df_2020['Type local']=='Appartement'
df_2020_habitation = df_2020[Maison_mask]
df_2020_habitation = pd.concat([df_2020_habitation,df_2020[Appart_mask]])

In [9]:
# fonction affichage d'une carte
def plot_fig(data, geo, colonne, proprietes, zoom, latitude,variable):
  fig = px.choropleth_mapbox(data, geojson=geo, locations=colonne, featureidkey=proprietes, color=variable, color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=zoom, center = {"lat": latitude, "lon": 2.351828},
                           opacity=0.5, width=1400, height = 800)
  fig.show()

In [ ]:
#affichage des prix au m² par départements pour les appartements et maison
plot_fig(df_2020_habitation, departements_france, 'Code departement', "properties.code", 5, 46.856578,'Prix au m2')

In [ ]:
#affichage des prix au m² par régions pour les appartements et maison
plot_fig(df_2020_habitation, regions_france, 'Regions', "properties.nom", 5, 46.856578,'Prix au m2')

In [ ]:
#Rapport valeurs foncières/nombre de piece par département
plot_fig(df_2020_habitation, departements_france, 'Code departement', "properties.code", 5, 46.856578,'Rapport valeurs foncières / nombre de pieces')

In [ ]:
#Rapport valeurs foncières/nombre de piece par régions
plot_fig(df_2020_habitation, regions_france, 'Regions', "properties.nom", 5, 46.856578,'Rapport valeurs foncières / nombre de pieces')

In [ ]:
#Calcul des prix au m² des maisons et appartement par commune
df_2020_prixm2_par_commune = df_2020_habitation[['Commune','Prix au m2']].groupby('Commune').mean()
df_2020_prixm2_par_commune['Commune'] = df_2020_prixm2_par_commune.index

In [ ]:
#prix au m2 des maisons et appartement par commune
plot_fig(df_2020_habitation, communes_france, 'Commune', "properties.nom", 5, 46.856578,'Prix au m2')

In [ ]:
#prix au m2 des local industriel et commercial
Local_mask = df_2020['Type local']=='Local industriel. commercial ou assimilé'
df_2020_local = df_2020[Local_mask]

In [ ]:
#Calcul des prix au m² des locaux industriels et commercial par région
plot_fig(df_2020_local, regions_france, 'Regions', "properties.nom", 5, 46.856578,'Prix au m2')

In [ ]:
#Calcul des prix au m² des locaux industriels et commercial par département
plot_fig(df_2020_local, departements_france, 'Code departement', "properties.code", 5, 46.856578,'Prix au m2')

In [ ]:
#Calcul des prix au m² des locaux industriels et commercial par ville
plot_fig(df_2020_local, communes_france, 'Commune', "properties.nom", 5, 46.856578,'Prix au m2')

In [ ]:
#moyenne des surfaces vendues par régions
plot_fig(df_2020, regions_france, 'Regions', "properties.nom", 5, 46.856578,'Surface terrain')

In [ ]:
#Calcul des prix au m² des locaux industriels et commercial par département
plot_fig(df_2020, departements_france, 'Code departement', "properties.code", 5, 46.856578,'Surface terrain')

In [ ]:
#Valeur foncière d'un culture nature par région
def look_by_nat_cult(data,attribut, nat_cult):
    data_natcult = data["Nature culture"]==nat_cult
    return data[data_natcult].groupby(attribut).mean()["Valeur fonciere"]
    
nature_culture = dict()
for k in df_2020.groupby('Nature culture').sum().axes[0]:
    nature_culture.update({k:look_by_nat_cult(df_2020, "Regions", k)})
nature_culture = pd.DataFrame(data=nature_culture)

def nc_val_fonc_by_reg(data, nat_cul):
    fig=px.bar(data[nat_cul], height=500, width=800,color=range(len(REGIONS.keys())),title=f"Valeur foncière d'une culture de nature {nat_cul} par région")
    fig.update_layout(yaxis_title="Valeur  Foncière")
    fig.show()
for k in df_2020.groupby('Nature culture').sum().axes[0]:
    nc_val_fonc_by_reg(nature_culture, k)


Evolution par Année

In [28]:
#import de l'ensemble des données
df_2017 = pd.read_csv('valeursfoncieres-2017.txt', sep="|",low_memory=False,decimal=',')
df_2018 = pd.read_csv('valeursfoncieres-2018.txt', sep="|",low_memory=False,decimal=',')
df_2019 = pd.read_csv('valeursfoncieres-2019.txt', sep="|",low_memory=False,decimal=',')
df_2020 = pd.read_csv('valeursfoncieres-2020.txt', sep="|",low_memory=False,decimal=',')
df_2021 = pd.read_csv('valeursfoncieres-2021.txt', sep="|",low_memory=False,decimal=',')

year = [2017, 2018, 2019, 2020, 2021]

df_2017 = ColonneVide(df_2017)
df_2018 = ColonneVide(df_2018)
df_2019 = ColonneVide(df_2019)
df_2020 = ColonneVide(df_2020)
df_2021 = ColonneVide(df_2021)


In [29]:
def addRegions(data):
    tab = []
    for i in data["Code departement"]:
        tab.append(find_key(REGIONS, i))
    data["Region"] = tab
    return data

In [30]:
df_2017 = addRegions(df_2017)
df_2018 = addRegions(df_2018)
df_2019 = addRegions(df_2019)
df_2020 = addRegions(df_2020)
df_2021 = addRegions(df_2021)

In [31]:

df_2017["Prix au m²"] = df_2017["Valeur fonciere"].div(df_2017["Surface terrain"], fill_value=1)
df_2018["Prix au m²"] = df_2018["Valeur fonciere"].div(df_2018["Surface terrain"], fill_value=1)
df_2019["Prix au m²"] = df_2019["Valeur fonciere"].div(df_2019["Surface terrain"], fill_value=1)
df_2020["Prix au m²"] = df_2020["Valeur fonciere"].div(df_2020["Surface terrain"], fill_value=1)
df_2021["Prix au m²"] = df_2021["Valeur fonciere"].div(df_2021["Surface terrain"], fill_value=1)




In [56]:
def plot_time_line_reg(col,region,title="",yaxis_title="", slide = False):
    
    
    T2017 = df_2017.loc[df_2017["Region"] == region]
    T2018 = df_2018.loc[df_2018["Region"] == region]
    T2019 = df_2019.loc[df_2019["Region"] == region]
    T2020 = df_2020.loc[df_2020["Region"] == region]
    T2021 = df_2021.loc[df_2021["Region"] == region]
    
    
    tmp = dict()
    tmp.update({col : [statistics.mean(T2017), T2018[['Prix au m²']].mean(), T2019[['Prix au m²']].mean(), T2020[['Prix au m²']].mean(), T2021[['Prix au m²']].mean()]})
    
    print(tmp)
    evo_reg = pd.DataFrame(data=tmp)
    
    fig = px.line(evo_reg,x=year, y=col ,width=700)
    fig.update_layout(title=(col if title=="" else title), xaxis_title="Date", yaxis_title=yaxis_title)
    if(slide):
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()


In [55]:
#Evolution du prix au M² par région

plot_time_line_reg('Prix au m²', "Normandie", "Evolution du prix au m²", "Prix au m²")

         No disposition Date mutation Nature mutation  Valeur fonciere  \
385168                1    07/01/2017           Vente          10000.0   
385169                1    12/01/2017           Vente          70000.0   
385170                1    12/01/2017           Vente          70000.0   
385171                1    12/01/2017           Vente          70000.0   
385172                1    12/01/2017           Vente          70000.0   
...                 ...           ...             ...              ...   
2548665               1    13/12/2017           Vente         110000.0   
2548666               1    21/12/2017           Vente          90000.0   
2548667               1    21/12/2017           Vente          90000.0   
2548668               1    21/12/2017           Vente          90000.0   
2548669               1    21/12/2017           Vente          90000.0   

         No voie B/T/Q Type de voie Code voie           Voie  Code postal  \
385168       3.0   NaN          RU